In [1]:
import networkx as nx
import numpy as np
import os
import veloxchem as vlx
import re
import time
import datetime
from tests.MOF_builder.functions.bbcif_properties import selected_type_vecs
from tests.MOF_builder.functions.place_bbs import superimpose
from tests.MOF_builder.functions.frag_recognizer import process_linker_molecule
from tests.MOF_builder.functions._readcif import  extract_type_atoms_fcoords_in_primitive_cell,read_cif,extract_atoms_fcoords_from_lines,extract_atoms_ccoords_from_lines
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.optimize import minimize
from tests.MOF_builder.functions.chainnode import process_chain_node,process_node
from tests.MOF_builder.functions.learn_template import extract_bridge_point_cluster_center_from_templatecif,extract_cluster_center_from_templatecif,make_supercell_3x3x3,find_pair_v_e,add_ccoords,set_DV_V,set_DE_E
from tests.MOF_builder.functions.learn_template import sort_nodes_by_type_connectivity,find_and_sort_edges_bynodeconnectivity
from tests.MOF_builder.functions.pillarstack_check import check_if_pillarstack,rotate_node_for_pillar
from tests.MOF_builder.functions.place_node_edge import addidx
from tests.MOF_builder.functions.node_rotation_matrix_optimizer import axis_optimize_rotations,apply_rotations_to_atom_positions,apply_rotations_to_xxxx_positions,update_ccoords_by_optimized_cell_params
from tests.MOF_builder.functions.place_node_edge import get_edge_lengths,update_node_ccoords,unit_cell_to_cartesian_matrix,cartesian_to_fractional,fractional_to_cartesian
from tests.MOF_builder.functions.place_node_edge import place_edgeinnodeframe
from tests.MOF_builder.functions.scale_cif_optimizer import optimize_cell_parameters

In [2]:
start_time = time.time()
linker_file = 'tests/triph.xyz'
molecule = vlx.Molecule.read_xyz_file(linker_file)
linker_topic = 3
center_frag_nodes_num,center_Xs,single_frag_nodes_num,frag_Xs= process_linker_molecule(molecule,linker_topic)

center is a cycle
[32, 27, 28, 29, 30, 31]
tritopic/tetratopic/multitopic: center is a cycle
find connected X in edge frag 0 3 32
find connected X in edge frag 9 6 28
find connected X in edge frag 18 21 30
center_frag: 9 [0, 5, 7]
outer_frag: 10 [4, 7]
nodes/tricenter.cif is writen
edges/triedge.cif is writen


In [3]:
node_cif = '21Alchain.cif'
linker_center_cif = 'edges/2B.cif'
linker_cif = 'edges/triedge.cif'
#linker
linker_x_vecs = selected_type_vecs(linker_cif,'.','X',False)
linker_length = np.linalg.norm(linker_x_vecs[0]-linker_x_vecs[1])
linker_cell_info,_, linker_atom_site_sector = read_cif(linker_cif)
ll,linker_atom, linker_ccoords = extract_atoms_ccoords_from_lines(linker_cell_info,linker_atom_site_sector)
print(linker_length,'linker_length')
#chainnode

node_target_type = 'Al'
node_unit_cell,node_atom, node_x_vecs, node_fcoords = process_node(node_cif, node_target_type)



2.779727434479862 linker_length
 no symmetry operation
 no symmetry operation


In [4]:
#template cif 

template_cif_file ='cau-1.cif'
cluster_distance_threshhold = 0.1
vvnode,cell_info,unit_cell = extract_bridge_point_cluster_center_from_templatecif(template_cif_file, 'Al',2,3.6) 
ecnode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'ZZ',1,0.1) # edge com in template cif file, use fcluster to find the cluster and center of the cluster
eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'C',6,1.45) # edge com in template cif file, use fcluster to find the cluster and center of the cluster

vvnode = np.unique(np.array(vvnode,dtype=float),axis=0)
ecnode = np.unique(np.array(ecnode,dtype=float),axis=0)
eenode = np.unique(np.array(eenode,dtype=float),axis=0)
##loop over super333xxnode and super333yynode to find the pair of x node in unicell which pass through the yynode
vvnode333 = make_supercell_3x3x3(vvnode)
ecnode333 = make_supercell_3x3x3(ecnode)
eenode333 = make_supercell_3x3x3(eenode)
#pair_vv_e,_,G=find_pair_v_e(vvnode333,eenode333)
#G = add_ccoords(G,unit_cell)
#G = set_DV_V(G)
#G = set_DE_E(G)

apply 16  symmetry operation
apply 16  symmetry operation
apply 16  symmetry operation


In [5]:
def check_inside_unit_cell(point):
    return all([i>=-0.01 and i<=1.01 for i in point])

def find_pair_v_e_c(vvnode333, ecnode333,eenode333): #exist center of linker  in mof 
    G = nx.Graph()
    pair_ve = []
    for e in eenode333:
        #print(e,'check')
        dist_v_e = np.linalg.norm(vvnode333 - e, axis=1)
        # find two v which are nearest to e, and at least one v is in [0,1] unit cell
        v1 = vvnode333[np.argmin(dist_v_e)]
        v1_idx = np.argmin(dist_v_e)
        dist_c_e = np.linalg.norm(ecnode333 - e, axis=1)
        # find two v which are nearest to e, and at least one v is in [0,1] unit cell
        v2 = ecnode333[np.argmin(dist_c_e)]
        v2_idx = np.argmin(dist_c_e)
        #print(v1,v2,'v1,v2')

        # find the center of the pair of v
        center = (v1 + v2) / 2
        # check if there is a v in [0,1] unit cell
        if check_inside_unit_cell(v1) or check_inside_unit_cell(v2):
            # check if the center of the pair of v is around e
            #if abs(np.linalg.norm(v1 - e)+np.linalg.norm(v2 - e) - np.linalg.norm(v1 - v2))< 1e-2: #v1,v2,e are colinear
            if np.linalg.norm(center - e) < 0.1:
                print(e,v1,v2,'check')
                G.add_node('V'+str(v1_idx), fcoords=v1,note='V')
                G.add_node('CV'+str(v2_idx), fcoords=v2,note ='CV')
                G.add_edge('V'+str(v1_idx), 'CV'+str(v2_idx), fcoords=(v1, v2),fc_center=e),
                pair_ve.append(('V'+str(v1_idx), 'CV'+str(v2_idx), e))
    return pair_ve, len(pair_ve), G
pair_vv_e,pair_num,G=find_pair_v_e_c(vvnode333,ecnode333,eenode333)
G = add_ccoords(G,unit_cell)
G = set_DV_V(G)
G = set_DE_E(G)

def sort_nodes_by_type_connectivity(G):
    Vnodes = [n for n in G.nodes() if G.nodes[n]['type']=='V']
    DVnodes = [n for n in G.nodes() if G.nodes[n]['type']=='DV']
    Vnodes = sorted(Vnodes,key=lambda x: G.degree(x),reverse=True)
    DVnodes = sorted(DVnodes,key=lambda x: G.degree(x),reverse=True)
    return Vnodes+DVnodes


sorted_nodes = sort_nodes_by_type_connectivity(G)

def check_edge_inunitcell(G,e):
    if 'DV' in G.nodes[e[0]]['type'] or 'DV' in G.nodes[e[1]]['type']:
        return False
    return True

def find_and_sort_edges_bynodeconnectivity(graph, sorted_nodes):
    all_edges = list(graph.edges())

    sorted_edges = []
    #add unit_cell edge first

    ei = 0
    while ei < len(all_edges):
            e = all_edges[ei]
            if check_edge_inunitcell(graph,e):
                sorted_edges.append(e)
                all_edges.pop(ei)
            ei += 1
    #sort edge by sorted_nodes
    for n in sorted_nodes:
        ei = 0
        while ei < len(all_edges):
            e = all_edges[ei]
            if n in e:
                if n ==e[0]:
                    sorted_edges.append(e)
                else:
                    sorted_edges.append((e[1],e[0]))
                all_edges.pop(ei)
            else:
                ei += 1

    return sorted_edges   
#fix one direction of the box, which should be parallel to the pilalar direction
#rotate the node to pillar direction and put all nodes into the cartesian coordinate 
#sorted_edges = find_and_sort_edges_bynodeconnectivity(G,sorted_nodes)

#firstly, check if all V nodes have highest connectivity
#secondly, sort all DV nodes by connectivity

[0.042 0.269 0.272] [0.092 0.227 0.5  ] [0.11  0.22  0.156] check
[0.042 0.731 0.727] [0.093 0.773 0.5  ] [0.11  0.78  0.844] check
[0.044 0.342 0.932] [0.273 0.408 1.   ] [-0.11   0.22   0.844] check
[0.044 0.659 0.068] [0.273 0.592 0.   ] [-0.11   0.78   0.156] check
[0.051 0.222 0.903] [0.273 0.408 1.   ] [-0.11   0.22   0.844] check
[0.051 0.778 0.097] [0.273 0.592 0.   ] [-0.11   0.78   0.156] check
[0.099 0.234 0.397] [0.092 0.227 0.5  ] [0.11  0.22  0.156] check
[0.099 0.766 0.603] [0.093 0.773 0.5  ] [0.11  0.78  0.844] check
[0.101 0.232 0.334] [0.092 0.227 0.5  ] [0.11  0.22  0.156] check
[0.101 0.768 0.666] [0.093 0.773 0.5  ] [0.11  0.78  0.844] check
[0.159 0.456 0.568] [0.092 0.227 0.5  ] [0.28  0.61  0.656] check
[0.159 0.544 0.432] [0.093 0.773 0.5  ] [0.28  0.39  0.344] check
[0.163 0.188 0.275] [0.092 0.227 0.5  ] [0.11  0.22  0.156] check
[0.163 0.813 0.724] [0.093 0.773 0.5  ] [0.11  0.78  0.844] check
[0.188 0.163 0.724] [0.227 0.093 0.5  ] [0.22  0.11  0.844] chec

In [6]:
sorted_edges = find_and_sort_edges_bynodeconnectivity(G,sorted_nodes)

node_xcoords = np.dot(node_unit_cell,node_x_vecs.T).T
node_coords = np.dot(node_unit_cell,node_fcoords.T).T

ec_unit_cell,ec_atom, ec_x_vecs, ec_fcoords = process_node(linker_center_cif, 'X')

ec_xcoords = np.dot(ec_unit_cell,ec_x_vecs.T).T
ecoords = np.dot(ec_unit_cell,ec_fcoords.T).T

nodexxxx = []
xxxx_positions_dict = {}
node_positions_dict = {}
chain_node_positions = []
#reindex the nodes in the xxxx_positions with the index in the sorted_nodes, like G has 16 nodes[2,5,7], but the new dictionary should be [0,1,2]
for n in sorted_nodes:
    if 'CV' in n:
        xxxx_positions_dict[sorted_nodes.index(n)]=addidx(G.nodes[n]['ccoords']+ec_xcoords) 
    else:
        xxxx_positions_dict[sorted_nodes.index(n)]=addidx(G.nodes[n]['ccoords']+node_xcoords) 

for n in sorted_nodes:
    if 'CV' in n:
        node_positions_dict[sorted_nodes.index(n)]=G.nodes[n]['ccoords']+ecoords
    else:
        node_positions_dict[sorted_nodes.index(n)]=G.nodes[n]['ccoords']+node_coords


#reindex the edges in the G with the index in the sorted_nodes
sorted_edges_of_sortednodeidx = [(sorted_nodes.index(e[0]),sorted_nodes.index(e[1])) for e in sorted_edges]


 no symmetry operation
 no symmetry operation


In [7]:
from tests.MOF_builder.functions.node_rotation_matrix_optimizer import optimize_rotations
def save_xyz(filename, rotated_positions_dict):
    """
    Save the rotated positions to an XYZ file for visualization.
    """
    with open(filename, "w") as file:
        num_atoms = sum(len(positions) for positions in rotated_positions_dict.values())
        file.write(f"{num_atoms}\n")
        file.write("Optimized structure\n")

        for node, positions in rotated_positions_dict.items():
            for pos in positions:
                file.write(f"X{node}   {pos[0]:.8f} {pos[1]:.8f} {pos[2]:.8f}\n")

# Optimize rotations
num_nodes = G.number_of_nodes()

###3D free rotation
optimized_rotations,static_xxxx_positions = optimize_rotations(num_nodes,G,sorted_nodes,sorted_edges_of_sortednodeidx, xxxx_positions_dict,maxfun=40000)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          648     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.38261D+07    |proj g|=  7.33417D+03


 This problem is unconstrained.



At iterate    1    f=  1.37951D+07    |proj g|=  3.17022D+03

At iterate    2    f=  1.37769D+07    |proj g|=  1.79410D+03

At iterate    3    f=  1.25835D+07    |proj g|=  5.52647D+02

At iterate    4    f=  1.25807D+07    |proj g|=  3.15346D+02

At iterate    5    f=  1.25796D+07    |proj g|=  1.57766D+02

At iterate    6    f=  1.25787D+07    |proj g|=  1.35228D+02

At iterate    7    f=  1.25779D+07    |proj g|=  1.56090D+02

At iterate    8    f=  1.14422D+07    |proj g|=  2.14092D+03

At iterate    9    f=  1.14400D+07    |proj g|=  9.91672D+02

At iterate   10    f=  1.14359D+07    |proj g|=  9.29087D+02

At iterate   11    f=  1.14274D+07    |proj g|=  9.14559D+02

At iterate   12    f=  1.14190D+07    |proj g|=  6.12251D+02

At iterate   13    f=  1.14176D+07    |proj g|=  5.11855D+02

At iterate   14    f=  1.14137D+07    |proj g|=  3.55393D+02

At iterate   15    f=  1.14102D+07    |proj g|=  2.73250D+02

At iterate   16    f=  1.14079D+07    |proj g|=  2.98396D+02

At iter

In [31]:
from tests.MOF_builder.functions.node_rotation_matrix_optimizer import apply_rotations_to_atom_positions
# Apply rotations
rotated_node_positions = apply_rotations_to_atom_positions(optimized_rotations, G, sorted_nodes,node_positions_dict)

# Save results to XYZ
save_xyz("optimized_nodesstructure.xyz", rotated_node_positions)


rotated_xxxx_positions_dict,optimized_pair=apply_rotations_to_xxxx_positions(optimized_rotations, G, sorted_nodes, sorted_edges_of_sortednodeidx,xxxx_positions_dict)


start_node = sorted_edges[0][0]#find_nearest_node_to_beginning_point(G)
#loop all of the edges in G and get the lengths of the edges, length is the distance between the two nodes ccoords
edge_lengths,lengths = get_edge_lengths(G)

constant_length = 1.2
x_com_length = np.mean([np.linalg.norm(i) for i in node_xcoords])
new_edge_length = linker_length+2*constant_length+2*x_com_length
#update the node ccoords in G by loop edge, start from the start_node, and then update the connected node ccoords by the edge length, and update the next node ccords from the updated node


updated_ccoords,original_ccoords = update_node_ccoords(G,edge_lengths,start_node,new_edge_length)
updated_ccoords,original_ccoords
#exclude the start_node in updated_ccoords and original_ccoords
updated_ccoords = {k:v for k,v in updated_ccoords.items() if k!=start_node}
original_ccoords = {k:v for k,v in original_ccoords.items() if k!=start_node}


#use optimized_params to update all of nodes ccoords in G, according to the fccoords

optimized_params = optimize_cell_parameters(cell_info,original_ccoords,updated_ccoords)
sG,scaled_ccoords = update_ccoords_by_optimized_cell_params(G,optimized_params)
scaled_node_positions_dict = {}
scaled_xxxx_positions_dict = {}

for n in sorted_nodes:
    if 'CV' in n:
        scaled_xxxx_positions_dict[sorted_nodes.index(n)]=addidx(sG.nodes[n]['ccoords']+ec_xcoords) 
    else:
        scaled_xxxx_positions_dict[sorted_nodes.index(n)]=addidx(sG.nodes[n]['ccoords']+node_xcoords) 

for n in sorted_nodes:
    if 'CV' in n:
        scaled_node_positions_dict[sorted_nodes.index(n)]=sG.nodes[n]['ccoords']+ecoords
    else:
        scaled_node_positions_dict[sorted_nodes.index(n)]=sG.nodes[n]['ccoords']+node_coords



# Apply rotations
scaled_rotated_node_positions  = apply_rotations_to_atom_positions(optimized_rotations, sG, sorted_nodes,scaled_node_positions_dict)
scaled_rotated_xxxx_positions,optimized_pair = apply_rotations_to_xxxx_positions(optimized_rotations, sG,sorted_nodes, sorted_edges_of_sortednodeidx, scaled_xxxx_positions_dict)
# Save results to XYZ
save_xyz("scale_optimized_nodesstructure.xyz", scaled_rotated_node_positions)

#placed_node,placed_edge = place_edgeinnodeframe(sorted_nodes,optimized_pair,node_atom,linker_atom,linker_x_vecs,linker_ccoords,scaled_rotated_xxxx_positions,scaled_rotated_chain_node_positions)

(2, 1)
[[ 8.98183873]
 [12.09577597]] 8.981838734037996
0 0 [[ 8.98183873]
 [12.09577597]]
V0 CV0 [0, 0]
(2, 1)
[[9.4081584 ]
 [9.84724013]] 9.408158403612292
0 0 [[9.4081584 ]
 [9.84724013]]
V1 CV1 [0, 0]
(1, 2)
[[9.70881962 9.09367097]] 9.093670968890216
0 1 [[9.70881962 9.09367097]]
CV4 V4 [0, 1]
(2, 1)
[[6.15460283]
 [8.27308883]] 6.154602830956645
0 0 [[6.15460283]
 [8.27308883]]
V2 CV8 [0, 0]
(2, 1)
[[8.16285102]
 [6.18143299]] 6.181432991949341
1 0 [[8.16285102]
 [6.18143299]]
V3 CV3 [1, 0]
(1, 2)
[[9.9301441  9.87741209]] 9.877412087023457
0 1 [[9.9301441  9.87741209]]
CV6 V12 [0, 1]
(1, 2)
[[13.21057843 10.28027388]] 10.280273878492315
0 1 [[13.21057843 10.28027388]]
CV7 V13 [0, 1]
(2, 1)
[[5.74535895]
 [6.66235389]] 5.745358952252385
0 0 [[5.74535895]
 [6.66235389]]
V12 CV12 [0, 0]
(1, 2)
[[11.97652953 14.35495806]] 11.976529532995775
0 0 [[11.97652953 14.35495806]]
CV10 V15 [0, 0]
(1, 2)
[[16.65248249 18.14700041]] 16.65248249400159
0 0 [[16.65248249 18.14700041]]
CV11 V14 [

In [72]:
def is_list_A_in_B(A,B):
        return all([np.allclose(a,b,atol=0.05) for a,b in zip(A,B)])

from tests.MOF_builder.functions.adjust_edges import  fetch_X_atoms_ind_array
def place_edgeinnodeframe(sG,sorted_nodes,optimized_pair,nodes_atom,linker_atom,linker_x_vecs,linker_ccoords,scaled_rotated_xxxx_positions,scaled_rotated_node_positions):
    
    linker_middle_point = np.mean(linker_x_vecs,axis=0)
    linker_xx_vec = linker_x_vecs - linker_middle_point
    norm_linker_xx_vec = linker_xx_vec/np.linalg.norm(linker_xx_vec)
    translated_linker_coords = linker_ccoords - linker_middle_point
    norm_xx_vector_record = []
    rot_record = []
    edges = {}
    for (i,j),pair in optimized_pair.items():
        x_idx_i,x_idx_j = pair
        reindex_i = sorted_nodes.index(i)
        reindex_j = sorted_nodes.index(j)
        x_i = scaled_rotated_xxxx_positions[reindex_i][x_idx_i][1:]
        x_j = scaled_rotated_xxxx_positions[reindex_j][x_idx_j][1:]
        x_i_x_j_middle_point = np.mean([x_i,x_j],axis=0)
        xx_vector = np.vstack([x_i-x_i_x_j_middle_point,x_j-x_i_x_j_middle_point])
        norm_xx_vector = xx_vector/np.linalg.norm(xx_vector)
        norm_xx_vector = np.round(xx_vector,6)
        #print(i,j,reindex_i,reindex_j,x_idx_i,x_idx_j)
        #use superimpose to get the rotation matrix
        #use record to record the rotation matrix for get rid of the repeat calculation
        indices = [index for index, value in enumerate(norm_xx_vector_record) if is_list_A_in_B(norm_xx_vector, value)]
        if len(indices) == 1: 
            rot = rot_record[indices[0]]
            #rot = reorthogonalize_matrix(rot)
        else:
            _, rot, _ = superimpose(norm_linker_xx_vec,norm_xx_vector)
            #rot = reorthogonalize_matrix(rot)
            norm_xx_vector_record.append(norm_xx_vector)
            rot_record.append(rot)
            
        #use the rotation matrix to rotate the linker x coords
        placed_edge_ccoords = np.dot(translated_linker_coords, rot) + x_i_x_j_middle_point
        placed_edge = np.hstack((np.asarray(linker_atom), placed_edge_ccoords))
        sG.edges[(i,j)]['coords'] = x_i_x_j_middle_point
        sG.edges[(i,j)]['points']=placed_edge
        _,sG.edges[(i,j)]['x_coords'] = fetch_X_atoms_ind_array(placed_edge,0,'X')
        edges[(i,j)]=placed_edge
    placed_node = {}
    for k,v in scaled_rotated_node_positions.items():
        #print(k,v)
        placed_node[k] = np.hstack((nodes_atom[k],v))
        sG.nodes[k]['points'] = np.hstack((nodes_atom[k],v))
        #find the atoms starts with "x" and extract the coordinates
        _,sG.nodes[k]['x_coords'] = fetch_X_atoms_ind_array(sG.nodes[k]['points'],0,'X')
    return placed_node,edges,sG

In [73]:
nodes_atoms = {}
for n in sorted_nodes:
    if 'CV' in n:
        nodes_atoms[n]= ec_atom
    else:
        nodes_atoms[n]= node_atom

placed_node,placed_edge,sG = place_edgeinnodeframe(sG,sorted_nodes,optimized_pair,nodes_atoms,linker_atom,linker_x_vecs,linker_ccoords,scaled_rotated_xxxx_positions,scaled_rotated_node_positions)

placed_all = []
frame_bb_node = []
with open("placed_structure.xyz", "w") as file:
    #node_num_atoms = sum(len(positions) for positions in scaled_rotated_chain_node_positions.values())
    node_num_atoms = sum(len(positions) for positions in placed_node.values())
    edge_num_atoms = sum(len(positions) for positions in placed_edge.values())
    num_atoms = node_num_atoms + edge_num_atoms
    file.write(f"{num_atoms}\n")
    file.write("Optimized structure\n")
    node_idx = 1
    edge_idx = -2
    for node, positions in placed_node.items():
        if 'DV' in G.nodes[node]['type']:
            sG.nodes[node]['index'] = 0
            continue
        for pos in positions:
            file.write(f"{pos[0]}   {pos[1]:.8f} {pos[2]:.8f} {pos[3]:.8f}\n")
            #if pos[0]=='X':
            #    pos[0] = 'C'
            line = np.array([pos[0],pos[1],pos[2],pos[3],0,pos[0],node_idx,'NODE'])
            placed_all.append(line)
            sG.nodes[node]['index'] = node_idx
            if 'CV' in node:
                continue
            else:
                frame_bb_node.append(line)
        node_idx += 1

    for edge,positions in placed_edge.items():
        
        #if 'DE' in G.edges[edge]['type']:
        #   # pass
        #    sG.edges[edge]['index'] = 0
        #    continue
        for pos in positions:
            file.write(f"{pos[0]}   {pos[1]:.8f} {pos[2]:.8f} {pos[3]:.8f}\n")
            #if pos[0]=='X':
               # pos[0] = 'C'
            line = np.array([pos[0],pos[1],pos[2],pos[3],0,pos[0],edge_idx,'EDGE'])
            placed_all.append(line)
            sG.edges[edge]['index'] = edge_idx
        edge_idx -= 1
            #placed_all.append(pos)



In [53]:
def nn(s):
    #no number
    return re.sub(r'\d+', '', s)

In [76]:
sG.edges('V0','CV0')

EdgeDataView([('V0', 'CV0', None), ('V0', 'CV5', None)])

In [95]:
sG.nodes['V0']['points'],sG.edges['V0','CV0']['points'],sG.degree['V0'],list(sG.neighbors('V0')),list(sG.neighbors('CV0'))
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import pdist, squareform
def find_pair_x_edge(x_matrix,edge_matrix):
    dist_matrix = np.zeros((len(x_matrix),len(edge_matrix)))
    for i in range(len(x_matrix)):
        for j in range(len(edge_matrix)):
            dist_matrix[i,j] = np.linalg.norm(x_matrix[i]-edge_matrix[j])
    row_ind, col_ind = linear_sum_assignment(dist_matrix)
    return row_ind,col_ind

#nodes connected to V0
#extract CV node Xatom
multiedge_orders = []
for n in sG.nodes:
    if 'CV' in n and sG.nodes[n]['type']=='V':
        edges =[]
        for con_n in list(sG.neighbors(n)):
            edges.append(sG.edges[n,con_n]['coords'])
        edges = np.vstack(edges,dtype=float)
        #extract the X atom in the CV node
        cv_xatoms = np.asarray(sG.nodes[n]['x_coords'][:,1:4],dtype=float)
        if len(cv_xatoms) < len(edges):
            #duplicate the cv_xatoms
            cv_xatoms = np.vstack([cv_xatoms]*len(edges))
        _,order = find_pair_x_edge(cv_xatoms,edges)
        con_n_order = [list(sG.neighbors(n))[i] for i in order]
        multiedge_orders.append((n,con_n_order))
        #make dist matrix of each x to each edge and then use hugerian algorithm to find the pair of x and edge
print(multiedge_orders)     
    
        

            



[('CV0', ['V0', 'V42']), ('CV1', ['V1', 'V187']), ('CV5', ['V0', 'V149']), ('CV4', ['V1', 'V4']), ('CV2', ['V215', 'V2']), ('CV3', ['V3', 'V54']), ('CV6', ['V150', 'V12']), ('CV7', ['V7', 'V13']), ('CV8', ['V8', 'V2']), ('CV9', ['V153', 'V3']), ('CV10', ['V15', 'V154']), ('CV11', ['V14', 'V11']), ('CV12', ['V12', 'V73']), ('CV13', ['V13', 'V200']), ('CV14', ['V14', 'V164']), ('CV15', ['V15', 'V21'])]


In [ ]:
#make dist matrix of x_matix to edge_matrix and then use hugerian algorithm to find the pair of x and edge
def find_pair_x_edge(x_matrix,edge_matrix):
    dist_matrix = pdist(np.vstack([x_matrix,edge_matrix]))
    dist_matrix = squareform(dist_matrix)
    row_ind, col_ind = linear_sum_assignment(dist_matrix)
    return row_ind,col_ind


In [ ]:
def sort_points_followXorder(points,Xpoints,sc_unit_cell):
    if len(Xpoints)==1: #this is only for linker center is only one point
        order = list(range(len(points)))
        return order
        
    order = []
    for x in Xpoints:
        dists=[]
        for p in range(len(points)):
            fvec = points[p]-x
            cvec = np.dot(sc_unit_cell,fvec)
            cdist = np.linalg.norm(cvec)
            dists.append(cdist)
        idx=dists.index(min(dists))
        if idx not in order:
            order.append(idx)
        else:
            print(x,idx,order,dists)
            raise ValueError("find more than one edge near center x ")
    print('edgeorder',order)
    return order

In [11]:
placed_all = np.array(placed_all)
placed_N = placed_all[placed_all[:,7]=='NODE']
placed_E = placed_all[placed_all[:,7]=='EDGE']

In [12]:
def limit_x(x):
    while x>0.5:
        x=x-1
    while x< -0.5:
        x = x+1
    return x

def fetch_node_withidx(placed_node,idx_list):
    res_id_list = [i+1 for i in idx_list ]
    res=[]
    for n in res_id_list:
        res.append(placed_node[placed_node[:,6]==n])
    return np.vstack(res)

def fetch_node_by_idx(placed_node,idx_list):
    res=[]
    for n in idx_list:
        res.append(placed_node[placed_node[:,6]==n])
    return np.vstack(res)
    
def fetch_edge_withidx(placed_edge,idx_list):
    res_id_list = [-1*i-1 for i in idx_list ]
    res=[]
    for n in res_id_list:
        res.append(placed_edge[placed_edge[:,6]==n])
    return np.vstack(res)

def fetch_edge_withidx_sep(placed_edge,idx_list):
    res_id_list = [-1*i-1 for i in idx_list ]
    res=[]
    for n in res_id_list:
        res.append(placed_edge[placed_edge[:,6]==n])
    return res

def fetch_edge_by_idx(placed_edge,idx_list):
    res=[]
    for n in idx_list:
        res.append(placed_edge[placed_edge[:,6]==n])
    return res

def sort_points_followXorder(points,Xpoints,sc_unit_cell):
    if len(Xpoints)==1: #this is only for linker center is only one point
        order = list(range(len(points)))
        return order
        
    order = []
    for x in Xpoints:
        dists=[]
        for p in range(len(points)):
            fvec = points[p]-x
            cvec = np.dot(sc_unit_cell,fvec)
            cdist = np.linalg.norm(cvec)
            dists.append(cdist)
        idx=dists.index(min(dists))
        if idx not in order:
            order.append(idx)
        else:
            print(x,idx,order,dists)
            raise ValueError("find more than one edge near center x ")
    print('edgeorder',order)
    return order

def centerize_edges_fc(target_edges_list,target_node_c_fc,target_nodes,sc_unit_cell):
    edges_update=[]
    edges_c_update=[]
    ordered_edges=[]
    target_nodes_X_fc=target_nodes[[row[5].startswith('X') for row in target_nodes]][:,1:4]
    for te_fcord in target_edges_list:
        te_fvec = te_fcord[:,1:4]- target_node_c_fc
        edge_c_fvec = np.mean(te_fvec,axis=0).tolist()
        cx,cy,cz = edge_c_fvec
        cx1 = limit_x(cx)
        cy1 = limit_x(cy)
        cz1 = limit_x(cz)
        differ = np.asarray([cx1,cy1,cz1])-np.asarray(edge_c_fvec)
        te_update = np.hstack((te_fcord[:,0:1],te_fvec+differ+target_node_c_fc,te_fcord[:,4:]))
        edges_update.append(te_update)
        edges_c_update.append(np.mean(te_fvec+differ+target_node_c_fc,axis=0))
    
    edges_order=sort_points_followXorder(edges_c_update, target_nodes_X_fc,sc_unit_cell)

    for k in edges_order:
            ordered_edges.extend(edges_update[k])
    return np.vstack((ordered_edges))

def merge_multitopic_node_edge_fc(TG,sc_unit_cell,multi_node_name,placed_nodes_arr_fc,placed_edges_arr_fc):
    multitopics=[]
    merge_track = []
    #edges_dict_list=list(TG.edges(data=True,keys=True)) #TODO:debug, this is original code
    
    for c_node in multi_node_name:
        c_node_idx=TG.nodes[c_node]['index']
        linked_multitopic = []
        #c_node connected edges
        connected_edges = [e for e in TG.edges(c_node)]
        for i in connected_edges:
            linked_multitopic.append(TG.edges[i]['index'])
        multitopics.append((c_node,c_node_idx,linked_multitopic))
    multitopic_edges = []
    for i in range(len(multitopics)):
        c_node = multitopics[i]
        node_idx = [c_node[1]]
        linked_edge_idx = c_node[2]
        #target_nodes = fetch_node_withidx(placed_nodes_arr_fc,node_idx) #TODO:debug, this is original code
        target_nodes = fetch_node_by_idx(placed_nodes_arr_fc,node_idx)
        target_nodes_c_fc = np.mean(target_nodes[:,1:4],axis=0)
        moded_trans_fc = np.mod(target_nodes_c_fc,1) - target_nodes_c_fc
        target_nodes[:,1:4] = target_nodes[:,1:4] + moded_trans_fc

        #target_edge_list = fetch_edge_withidx_sep(placed_edges_arr_fc,linked_edge_idx) #TODO:debug, this is original code
        target_edge_list = fetch_edge_by_idx(placed_edges_arr_fc,linked_edge_idx)
        target_nodes_c_fc = np.mean(target_nodes[:,1:4],axis=0)
        target_edges = centerize_edges_fc(target_edge_list,target_nodes_c_fc,target_nodes,sc_unit_cell)
        multitopic_edge = np.vstack((target_nodes,target_edges))

        for row_n in range(len(multitopic_edge)):
            multitopic_edge[row_n,5]= re.sub('[0-9]','',multitopic_edge[row_n,5]) + str(row_n+1)
        multitopic_edge[:,6]=[-1*i-2]*len(multitopic_edge)
        multitopic_edge[:,7]=['EDGE']*len(multitopic_edge)
        multitopic_edges.append(multitopic_edge)

        merge_track.append((-1*i-2,node_idx,linked_edge_idx))

    multitopic_edges_fcoords = np.vstack(multitopic_edges)

    return multitopic_edges_fcoords,merge_track

In [13]:
multi_node_name = []
for n in sorted_nodes:
    if G.nodes[n]['type']=='V' in n and G.nodes[n]['note']=='CV':
        multi_node_name.append(n)
print('multi_node_name',multi_node_name)

sc_unit_cell = unit_cell_to_cartesian_matrix(optimized_params[0],optimized_params[1],optimized_params[2],optimized_params[3],optimized_params[4],optimized_params[5])

multi_node_name ['CV0', 'CV1', 'CV5', 'CV4', 'CV2', 'CV3', 'CV6', 'CV7', 'CV8', 'CV9', 'CV10', 'CV11', 'CV12', 'CV13', 'CV14', 'CV15']


In [14]:
def locate_min_idx(matrix):
        min_idx = np.unravel_index(matrix.argmin(), matrix.shape)
        return min_idx[0],min_idx[1]
#remove charater from string
def nl(s):
    return int(re.sub('a-zA-Z','',s))

def calculate_primitive_eG_net_multitopic(sG):
        #calculate and add all edge_center as node to eG, then search for node_center in a range(around closest node distance)
        #this eG is searching neighbor nodes from edge, so the absolute isolated nodes (just single node) cannot be counted because it cannot be found from an edge 
        #eG nodes are all nodes with note CV in TG
        eG=nx.Graph()  
        for n in list(sG.nodes()):
            if sG.nodes[n]['note']=='CV':
                eG.add_node('E'+ str(sG.nodes[n]['index'],[0,0,0]),ccoords= sG.nodes[n]['ccoords'],fcoords= sG.nodes[n]['fcoords'],type= 'edge_center',points = )
                #add all connected nodes of CV-node to eG
                for i in list(sG.neighbors(n)):
                    if sG.nodes[i]['type']=='V': #not DV
                        eG.add_node((sG.nodes[i]['index'],[0,0,0]),ccoords= sG.nodes[i]['ccoords'],fcoords= sG.nodes[i]['fcoords'],type= 'node')
                        eG.add_edge('E'+str(sG.nodes[n]['index'],[0,0,0]),sG.nodes[i]['index'],type='real')

        nodefc_centers = [(sG.nodes[ne]['index'],sG.nodes[ne]['fcoords'],sG.nodes[ne]['ccoords']) for ne in sG.nodes() if sG.nodes[ne]['type']=='V' and sG.nodes[ne]['note']=='V']


        #add virtual edges
        n_n_distance_matrix = np.zeros((len(nodefc_centers),len(nodefc_centers)))
        for i in range(len(nodefc_centers)):
            for j in range(len(nodefc_centers)):
                n_n_distance_matrix[i,j] = np.linalg.norm(nodefc_centers[i][1] - nodefc_centers[j][1]) #distance between two nodes, ccoords
            n_n_distance_matrix[i,i] = 1000
        #use hungrain algorithm to find the shortest path between all nodes
        

        for i in range(len(nodefc_centers)):
            n_n_min_distance = np.min(n_n_distance_matrix[i:i+1,:])
            if n_n_min_distance < 0.6:
                _,n_j = locate_min_idx(n_n_distance_matrix[i:i+1,:])
                eG.add_edge((nodefc_centers[i][0],[0,0,0]),(nodefc_centers[n_j][0],[0,0,0]),type='virtual')
                print('add virtual edge',(nodefc_centers[i][0],[0,0,0]),(nodefc_centers[n_j][0],[0,0,0]))
        return eG

In [15]:
from tests.MOF_builder.functions.adjust_edges import adjust_edges,addxoo2edge,superimpose
from tests.MOF_builder.functions.write_cifs import write_cif_nobond, merge_catenated_cifs #write_check_cif,bond_connected_components, distance_search_bond, fix_bond_sym, merge_catenated_cifs
#from functions.scale_animation import scaling_callback_animation, write_scaling_callback_animation, animate_objective_minimization
import itertools
#from random import choice
from tests.MOF_builder.functions.cluster import placed_arr#cluster_supercell,
from tests.MOF_builder.functions.supercell import find_new_node_beginning,Carte_points_generator
from tests.MOF_builder.functions.output import tempgro,temp_xyz,viewgro#,viewxyz
from tests.MOF_builder.functions.terminations import terminate_nodes,terminate_unsaturated_edges,add_node_terminations,exposed_xoo_cc,Xpdb,terminate_unsaturated_edges_CCO2
#from functions.filtX import filt_nodex_fvec,filt_closest_x_angle,filt_outside_edgex
#unctions.multitopic import 
from tests.MOF_builder.functions.isolated_node_cleaner import reindex_frag_array,get_frag_centers_fc,calculate_eG_net_ditopic,calculate_eG_net_ditopic_PILLAR,calculate_eG_net
from tests.MOF_builder.functions.replace import fetch_by_idx_resname,sub_pdb

def filter_connected_node_loose(sG,bare_nodeedge_fc_loose,boundary_node_res_loose):
    renode_fcarr=reindex_frag_array(bare_nodeedge_fc_loose,'NODE')
    reedge_fcarr=reindex_frag_array(bare_nodeedge_fc_loose,'EDGE')
    node_fcarr = bare_nodeedge_fc_loose[bare_nodeedge_fc_loose[:,4]== 'NODE']
    edge_fcarr = bare_nodeedge_fc_loose[bare_nodeedge_fc_loose[:,4]== 'EDGE']

    edgefc_centers = get_frag_centers_fc(reedge_fcarr)
    nodefc_centers = get_frag_centers_fc(renode_fcarr)

    eG = calculate_eG_net_multitopic(sG)
    print(eG.nodes())
    #nodes_indices = [i+1 for i in range(nodefc_centers[-1][0])]
    connected_nodes_indices=[i for i in list(eG.nodes) if isinstance(i,int)] # the name of nodes in eG is int, edges' are Exx (str)
    #isolated_nodes_indices = [i for i in nodes_indices if i not in connected_nodes_indices]

    mask = np.isin(node_fcarr[:, 5], connected_nodes_indices)
    connected_nodes_fcarr = node_fcarr[mask]
    connected_nodeedge_fc_loose = np.vstack((connected_nodes_fcarr,edge_fcarr)) ##

    nodes_fc=bare_nodeedge_fc_loose[bare_nodeedge_fc_loose[:,4]=='NODE']
    mask_boundary_nodefc=np.isin(nodes_fc[:,5],boundary_node_res_loose)
    print(mask_boundary_nodefc,"mask")

    boundary_connected_nodes_res=np.unique(node_fcarr[mask_boundary_nodefc & mask][:,5]) ##

    return connected_nodeedge_fc_loose, boundary_connected_nodes_res,eG

def 

from cluster import supercell_nodeedge_fc_loose_check,filt_boundary_res_loose_check

def cluster_supercell(sG,sc_unit_cell,supercell_Carte,linker_topics,target_all_fc,box_bound,scalar,cutx,cuty,cutz,boundary_scalar):
    s_fvec_all_loose,row_diff_idx_loose = supercell_nodeedge_fc_loose_check(supercell_Carte,target_all_fc,box_bound,scalar,cutx,cuty,cutz) 
    print("row_diff_idx_loose",row_diff_idx_loose)
    safe_res_fc_loose,extra_res_fc_loose,boundary_node_res_loose = filt_boundary_res_loose_check(s_fvec_all_loose,row_diff_idx_loose,box_bound,scalar,cutx,cuty,cutz,boundary_scalar)
    print(boundary_node_res_loose,"boundary node res loose")
    bare_nodeedge_fc_loose=np.vstack((safe_res_fc_loose,extra_res_fc_loose))
    connected_nodeedge_fc_loose, boundary_connected_nodes_res,eG = filter_connected_node_loose(sG,bare_nodeedge_fc_loose,boundary_node_res_loose)
    return connected_nodeedge_fc_loose, boundary_connected_nodes_res,eG,bare_nodeedge_fc_loose


class MOF_multitopic:
    #def __init__(self,template,node,edge,node_topics):
    #    self.template = template
    #    self.node = node
    #    self.edge = edge
    #    self.linker_topics = 2 #ditopic class 
    #    self.node_topics = node_topics
	
    def __init__(self):
        self.linker_topics = 2
        pass
    def load(self,TG,placed_all,sc_unit_cell,placed_nodes,placed_edges,multi_node_name,frame_nodes):
        self.TG = TG
        self.placed_all = placed_all
        self.sc_unit_cell = sc_unit_cell
        self.placed_nodes = placed_nodes
        self.placed_edges = placed_edges
        self.tetra_node_name = multi_node_name
        self.frame_nbb_node	= frame_nodes

    
    def basic_supercell(self, supercell, term_file='data/methyl.pdb', boundary_cut_buffer=0.00, edge_center_check_buffer=0.0, cutxyz=[True, True, True]):
        # Generate a basic supercell with terminations
        linker_topics = self.linker_topics
        scalar = boundary_cut_buffer
        boundary_scalar = edge_center_check_buffer
        cutx, cuty, cutz = cutxyz
        TG = self.TG     
        tetra_node_name = self.tetra_node_name
        placed_edges = self.placed_edges
        placed_nodes = self.placed_nodes
        sc_unit_cell = self.sc_unit_cell
        frame_nbb_node = self.frame_nbb_node		
        frame_node_name = [i for i in list(TG.nodes()) if i not in tetra_node_name]
        frame_node_fc = np.asarray([TG.nodes[fn]['fcoords'] for fn in frame_node_name])
    
        placed_nodes_arr, nodes_id = placed_arr(placed_nodes)
        placed_edges_arr, edges_id = placed_arr(placed_edges)		
        placed_nodes_fc = np.hstack((placed_nodes_arr[:,0:1], (np.dot(np.linalg.inv(sc_unit_cell), placed_nodes_arr[:,1:4].T)).T, placed_nodes_arr[:,4:]))
        placed_edges_fc = np.hstack((placed_edges_arr[:,0:1], (np.dot(np.linalg.inv(sc_unit_cell), placed_edges_arr[:,1:4].T)).T, placed_edges_arr[:,4:]))	
        placed_frame_node, _ = placed_arr(np.c_[frame_nbb_node, ['NODE']*len(frame_nbb_node)])
        placed_frame_node_fc = np.hstack((placed_frame_node[:,0:1], (np.dot(np.linalg.inv(sc_unit_cell), placed_frame_node[:,1:4].T)).T, placed_frame_node[:,4:-1]))		
        
        tetratopic_edges_fcoords,merge_track = merge_multitopic_node_edge_fc(TG,sc_unit_cell,tetra_node_name,placed_nodes_fc,placed_edges_fc)	
 
        #target_all_fc = np.vstack((placed_frame_node_fc,tetratopic_edges_fcoords)) #TODO:debug, this is original code
        target_all_fc = np.vstack((placed_nodes_fc,placed_nodes_fc))
        #target_all_fc = np.vstack((placed_nodes_fc,tetratopic_edges_fcoords)) # the reason of using above version node is because we need xoo in node for terminations adding
        box_bound= supercell+1
        supercell_Carte = Carte_points_generator(supercell)		

        connected_nodeedge_fc, boundary_connected_nodes_res, eG, bare_nodeedge_fc_loose = cluster_supercell(TG,sc_unit_cell, supercell_Carte, linker_topics, target_all_fc, box_bound, scalar, cutx, cuty, cutz, boundary_scalar)		
        terms_cc_loose = terminate_nodes(term_file, boundary_connected_nodes_res, connected_nodeedge_fc, sc_unit_cell, box_bound)

        connected_nodeedge_cc = np.hstack((connected_nodeedge_fc[:,:-3], (np.dot(sc_unit_cell, connected_nodeedge_fc[:,-3:].T)).T))
        node_edge_term_cc_loose = np.vstack((connected_nodeedge_cc, terms_cc_loose))		
        self.all_connected_node_edge_cc = connected_nodeedge_cc
        self.all_terms_cc_loose = terms_cc_loose
        self.all_N_E_T_cc = node_edge_term_cc_loose		
        self.bare_nodeedge_fc = bare_nodeedge_fc_loose
        #self.eG = eG
        self.nodes_id = nodes_id
        self.edges_id = edges_id
		
    def write_basic_supercell(self,gro,xyz):
        #tempgro('40test.gro',self.all_connected_node_edge_cc[self.all_connected_node_edge_cc[:,5]==1])
        tempgro(gro,self.all_connected_node_edge_cc)
        temp_xyz(xyz,self.all_connected_node_edge_cc)
        #viewgro("40test.gro")
        viewgro(gro)
	
	
    '''
        #Defective model: node/edge missing
            #delete nodes or edges
            #terminate nodes ROUND1
            # find main fragment
                # find unstaturated node uN1
                    # find uN1 neighbors and extract X in neighbor edge(E+int)
                        # filt exposed X sites in uN1
                            # add terminations  
            #terminate edge ROUND2
                #find unsaturaed edge uE1
                    #find uE1 neighbors and extract X in neighbor node 'int'
                        # filt exposed X sites in uE1
                            # add terminations (-OH)    
        #Defective model: linker exchange
        #   termination OO don't change, use X to set a range 
        #   atoms in outX_range stay
        #    then superimpose by X  
            #superimpose for replacement
                #find X AND super impose
    '''



    def defect_missing(self,remove_node_list = [],remove_edge_list = []):
        bare_nodeedge_fc_loose = self.bare_nodeedge_fc
        linker_topics = self.linker_topics
        node_topics = self.node_topics
        sc_unit_cell = self.sc_unit_cell        
        renode1_fcarr = reindex_frag_array(bare_nodeedge_fc_loose, 'NODE')
        reedge1_fcarr = reindex_frag_array(bare_nodeedge_fc_loose, 'EDGE')
        defective_node_fcarr = np.vstack(([i for i in renode1_fcarr if i[5] not in remove_node_list]))
        defective_edge_fcarr = np.vstack(([i for i in reedge1_fcarr if i[5] not in remove_edge_list]))
        renode_fcarr = defective_node_fcarr
        reedge_fcarr = defective_edge_fcarr
        edgefc_centers = get_frag_centers_fc(reedge_fcarr)
        nodefc_centers = get_frag_centers_fc(renode_fcarr)      
        
        eG = calculate_eG_net_multitopic(edgefc_centers,nodefc_centers,linker_topics,4)
        eG_subparts=[len(c) for c in sorted(nx.connected_components(eG), key=len, reverse=True)]
        #for pillar stack, add virtual edge to connect clostest nodes, set distance to 0.6
        if len(eG_subparts)>1:
            print(f'this MOF has {len(eG_subparts)} seperated fragments: {eG_subparts}')
        else:
            print(f'this MOF has {len(eG_subparts)} fragment')

        

        unsaturated_nodes = [(n, d) for n, d in eG.degree() if d < node_topics and isinstance(n, int)]
        unsaturated_edges = [(n, d) for n, d in eG.degree() if d < linker_topics and isinstance(n, str)]
        if len(unsaturated_edges) > 0:
            print(f"UNsaturated edges(linkers) exist, need linker_termination <= {len(unsaturated_edges)}")
        else:
            print("only saturated edges(linkers) exist")     
                  
        if len(unsaturated_nodes) > 0:
            print(f"UNsaturated nodes exist, <={len(unsaturated_nodes)} nodes need node_termination")
        else:
            print("only saturated nodes exist")

        frags = [(len(c), c) for c in sorted(nx.connected_components(eG), key=len, reverse=True)]
        main_frag = list(sorted(nx.connected_components(eG), key=len, reverse=True)[0])
        main_frag_nodes = [i for i in main_frag if isinstance(i, int)]
        main_frag_edges = [i for i in main_frag if re.sub('[0-9]', '', str(i)) == 'E']
        unsaturated_main_frag_nodes = [i for i in unsaturated_nodes if i[0] in main_frag_nodes]
        unsaturated_main_frag_edges = [i for i in unsaturated_edges if i[0] in main_frag_edges]      
        main_frag_half_edges_fc = np.vstack(([reedge_fcarr[reedge_fcarr[:,5] == int(ei[1:])] for ei in main_frag_edges]))
        main_frag_nodes_fc = np.vstack(([renode_fcarr[renode_fcarr[:,5] == ni] for ni in main_frag_nodes]))
        main_frag_edges_fc, xoo_dict, con_nodes_x_dict = addxoo2edge(eG, main_frag_nodes, main_frag_nodes_fc, main_frag_edges, main_frag_half_edges_fc, sc_unit_cell)

        main_frag_nodes_cc = np.hstack((main_frag_nodes_fc[:,:-3], np.dot(sc_unit_cell, main_frag_nodes_fc[:,-3:].T).T))
        main_frag_edges_cc = np.hstack((main_frag_edges_fc[:,:-3], np.dot(sc_unit_cell, main_frag_edges_fc[:,-3:].T).T))
        self.eG = eG
        self.main_frag_nodes = main_frag_nodes
        self.main_frag_edges = main_frag_edges
        self.unsaturated_main_frag_nodes = unsaturated_main_frag_nodes
        self.unsaturated_main_frag_edges = unsaturated_main_frag_edges
        self.node_xoo_dict = xoo_dict
        self.con_nodes_x_dict = con_nodes_x_dict
        self.main_frag_nodes_cc = main_frag_nodes_cc
        self.main_frag_edges_cc = main_frag_edges_cc   

    def term_defective_model(self, n_term_file='data/methyl.pdb', e_termfile='data/CCO2H2.pdb'):
        # Terminate the defective model with specified terminations
        eG = self.eG
        unsaturated_main_frag_nodes = self.unsaturated_main_frag_nodes
        main_frag_nodes = self.main_frag_nodes
        main_frag_nodes_cc = self.main_frag_nodes_cc
        con_nodes_x_dict = self.con_nodes_x_dict
        xoo_dict = self.node_xoo_dict
        unsaturated_main_frag_edges = self.unsaturated_main_frag_edges
        main_frag_edges_cc = self.main_frag_edges_cc
        linker_topics = self.linker_topics
		
		# get indices for cleaved node atoms in main_frag_nodes without xoo
        xoo_ind = []
        for key in list(xoo_dict):
            xoo_ind.append(key)
            xoo_ind += xoo_dict[key]
        single_node = main_frag_nodes_cc[main_frag_nodes_cc[:,5] == main_frag_nodes_cc[0,5]]
        node_nums = len(single_node)
        single_node_stay_ind = np.asarray([i for i in range(node_nums) if i not in xoo_ind])
        a = []
        for node in range(len(main_frag_nodes)):
            a.append(node_nums * node + single_node_stay_ind)
        metal_node_indices = np.hstack(a)

		# add -methyl to terminate nodes find exposed xoo 
        ex_node_cxo_cc_loose = exposed_xoo_cc(eG, unsaturated_main_frag_nodes, main_frag_nodes_cc, con_nodes_x_dict, xoo_dict)
        n_terms_loose = add_node_terminations(n_term_file, ex_node_cxo_cc_loose)
        if len(n_terms_loose) > 0:
            n_terms_cc = np.vstack((n_terms_loose))
            self.n_terms_cc = n_terms_cc
        else:
            n_terms_cc = np.empty((0,9))
            self.n_terms_cc = n_terms_cc
            
        
		# add -COOH term to exposed edge and change edge name to HEDGE
        cleaved_metal_node = main_frag_nodes_cc[metal_node_indices]
        t_edges = terminate_unsaturated_edges(e_termfile, unsaturated_main_frag_edges, eG, main_frag_edges_cc, linker_topics)
        node_edge_term_cc = np.vstack((cleaved_metal_node, t_edges, n_terms_cc))
        self.t_edges = t_edges
        self.tn_te_cc = node_edge_term_cc

    def write_tntemof(self, gro):
        # Write the terminated defective model to a file
        tempgro(gro, self.tn_te_cc)
        viewgro(gro)

    def defect_replace_linker(self, sub_file, sub_class, candidate_res_idx_list, sub_res_newname='SUB'):
        # Replace linkers in the defective model with a specified substitution
        node_edge_term_cc = self.tn_te_cc
        for res_idx in candidate_res_idx_list:
            fetch_res_mask, other_mask = fetch_by_idx_resname(node_edge_term_cc, res_idx, sub_class)
            other_res = node_edge_term_cc[other_mask]
            selected_res = node_edge_term_cc[fetch_res_mask]
            if len(selected_res) == 0:
                continue
            X_atoms_ind = [i for i in range(len(selected_res)) if selected_res[i,2][0] == 'X']
            outer_atoms_ind = [j for j in range(len(selected_res)) if (j > X_atoms_ind[0]) & (j not in X_atoms_ind)]
            stay_outer_atoms = selected_res[outer_atoms_ind]
            X_atoms = selected_res[X_atoms_ind]
            sub_data = sub_pdb(sub_file)
            subX, subX_ind = Xpdb(sub_data, 'X')
            subX_coords_cc = subX[:,-3:]
            subX_coords_cc = subX_coords_cc.astype('float')
            X_atoms_coords_cc = X_atoms[:,-3:]
            X_atoms_coords_cc = X_atoms_coords_cc.astype('float')
            self.subX_coords_cc = subX
            self.X_atoms_coords_cc = X_atoms

            _, rot, trans = superimpose(subX_coords_cc, X_atoms_coords_cc)
            sub_coords = sub_data[:,-3:]
            sub_coords = sub_coords.astype('float')
            placed_sub_data = np.hstack((sub_data[:,:-3], np.dot(sub_coords, rot) + trans))

            sub_edge = np.vstack((stay_outer_atoms, placed_sub_data))
            sub_edge[:,5] = stay_outer_atoms[0,5]
            sub_edge[:,4] = sub_res_newname
            for row in range(len(sub_edge)):
                sub_edge[row,2] = re.sub('[0-9]', '', sub_edge[row,2]) + str(row + 1)
            node_edge_term_cc = np.vstack((other_res, sub_edge))
        self.replaced_tn_te_cc = node_edge_term_cc

    def write_view_replaced(self, gro):
        # Write and view the replaced defective model
        tempgro(gro, self.replaced_tn_te_cc)
        viewgro(gro)






In [16]:
##nodes in sG type: V
#for n in sG.nodes():
#    if 'V' in sG.nodes[n]['type'] and 'CV' not in sG.nodes[n]['note']:
#        print(n,sG.nodes[n]['ccoords'],sG.nodes[n]['type'])
#

In [17]:
#clean DV node in sG
#for n in list(sG.nodes()):
#    if sG.nodes[n]['type'] == 'DV':
#        sG.remove_node(n)
#

In [18]:
n_term_file = 'data/terminations_database/methyl.pdb'
e_termfile = 'data/terminations_database/CCO2.pdb'
supercell = np.asarray([0,0,0])
testmof = MOF_multitopic()
testmof.load(sG,placed_all,sc_unit_cell,placed_N,placed_E,multi_node_name,frame_bb_node)

testmof.basic_supercell(np.asarray(supercell),term_file = 'data/terminations_database/methyl.pdb',boundary_cut_buffer = 0,edge_center_check_buffer = 0.0,cutxyz=[True,True,True])
testmof.write_basic_supercell('20test.gro','20test.xyz')
testmof.node_topics = 2
testmof.defect_missing()
testmof.term_defective_model(n_term_file=n_term_file,e_termfile=e_termfile)
testmof.write_tntemof('30.gro')    

#'''cleave all unsaturated linkers'''
u_edge_idx=[]
for i in testmof.unsaturated_main_frag_edges:
    u_edge_idx.append(int(i[0][1:]))
print(u_edge_idx)

testmof.defect_missing([],u_edge_idx)#+[26,20,18,30,8,34,33,4,28,29])
testmof.term_defective_model(n_term_file=n_term_file,e_termfile=e_termfile)
testmof.write_tntemof('33.gro') 

row_diff_idx_loose [18, 22, 23, 26, 46, 48, 51, 52, 54, 56, 58, 59, 62, 64, 65, 66, 69, 70, 72, 74, 76, 77, 80, 99, 103, 104, 107, 109, 110, 111, 114, 115, 136, 138, 141, 142, 156, 157, 160, 161, 162, 163, 164, 177, 178, 179, 182, 183, 184, 185, 239, 240, 241, 244, 245, 246, 247, 260, 261, 262, 265, 266, 267, 268, 408, 409, 410, 413, 414, 415, 416, 428, 429, 430, 433, 434, 435, 436, 449, 450, 451, 454, 455, 456, 457, 470, 471, 472, 475, 476, 477, 478]
[3, 6, 7, 8, 9, 12, 13, 16] boundary node res loose
add virtual edge 1 8
add virtual edge 3 10
add virtual edge 5 13
add virtual edge 8 1
add virtual edge 10 3
add virtual edge 12 19
add virtual edge 13 5
add virtual edge 15 22
add virtual edge 19 12
add virtual edge 20 26
add virtual edge 21 27
add virtual edge 22 15
add virtual edge 24 32
add virtual edge 26 20
add virtual edge 27 21
add virtual edge 32 24
['E2', 1, 'E4', 3, 'E0', 5, 'E6', 'E7', 13, 'E9', 8, 'E11', 10, 12, 'E14', 20, 'E16', 15, 21, 'E17', 19, 'E18', 22, 'E23', 27, 'E25'

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

TypeError: calculate_eG_net_multitopic() takes 1 positional argument but 4 were given

In [ ]:
ERROR
eG.add_node

In [ ]:
start_time = time.time()
linker_file = 'ndi.xyz'
molecule = vlx.Molecule.read_xyz_file(linker_file)
linker_topic =2
center_frag_nodes_num,center_Xs,single_frag_nodes_num,frag_Xs= process_linker_molecule(molecule,linker_topic)
#pillar stack structure:
#regarding chain_node cif file:
    #find the pillar direction vector which is defined by the Al-Al vectors in chain_node cif file 

#regarding the linker cif file:
    #find the length of the linker which should be the maximum length of X vectors in the linker cif file

#regarding the template cif file:
    #find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file, 


#the unit_cell of the template cif file should be scaled to fit the length of the linker, and the box_vector which along pillar direction should be fixed


chain_node_cif = '21Alchain.cif'
#template_cif = 'chain_rna.cif'
linker_cif = 'edges/diedge.cif'

#linker
linker_x_vecs = selected_type_vecs(linker_cif,'.','X',False)
#_,_, linker_x_vecs=extract_type_atoms_ccoords_in_primitive_cell(linker_cif, 'X')
#ditopic linker only has two x vectors
#linker_length = calc_edge_len(linker_cif,'.') #length of the edge should be x-x length in linker cif file, unit angstrom
linker_length = np.linalg.norm(linker_x_vecs[0]-linker_x_vecs[1])
linker_cell_info,_, linker_atom_site_sector = read_cif(linker_cif)

ll,linker_atom, linker_ccoords = extract_atoms_ccoords_from_lines(linker_cell_info,linker_atom_site_sector)
print(linker_length,'linker_length')


#chainnode

node_target_type = 'Al'
node_unit_cell,node_atom,node_pillar_fvec, node_x_vecs, chain_node_fcoords = process_chain_node(chain_node_cif, node_target_type)

#template cif 

template_cif_file ='MIL53templatecif.cif'
cluster_distance_threshhold = 0.1
vvnode,cell_info,unit_cell = extract_cluster_center_from_templatecif(template_cif_file, 'YY',1,1) # node com in template cif file, use fcluster to find cluster and the center of the cluster
eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'XX',1,1) # edge com in template cif file, use fcluster to find the cluster and center of the cluster

#loop over super333xxnode and super333yynode to find the pair of x node in unicell which pass through the yynode
vvnode333 = make_supercell_3x3x3(vvnode)
eenode333 = make_supercell_3x3x3(eenode)
pair_vv_e,_,G=find_pair_v_e(vvnode333,eenode333)
G = add_ccoords(G,unit_cell)
G = set_DV_V(G)
G = set_DE_E(G)
#debug
#check_connectivity(G)
#check_edge(G)
#firstly, check if all V nodes have highest connectivity
#secondly, sort all DV nodes by connectivity

sorted_nodes = sort_nodes_by_type_connectivity(G)
#fix one direction of the box, which should be parallel to the pilalar direction
#rotate the node to pillar direction and put all nodes into the cartesian coordinate 
sorted_edges = find_and_sort_edges_bynodeconnectivity(G,sorted_nodes)


PILLAR,pillar_vec = check_if_pillarstack(G) 
pillar_oriented_node_xcoords,pillar_oriented_node_coords = rotate_node_for_pillar(G,node_unit_cell,node_pillar_fvec,pillar_vec,node_x_vecs,chain_node_fcoords)

nodexxxx = []
xxxx_positions_dict = {}
chain_node_positions_dict = {}
chain_node_positions = []
#reindex the nodes in the xxxx_positions with the index in the sorted_nodes, like G has 16 nodes[2,5,7], but the new dictionary should be [0,1,2]
xxxx_positions_dict = {sorted_nodes.index(n):addidx(G.nodes[n]['ccoords']+pillar_oriented_node_xcoords) for n in sorted_nodes}
chain_node_positions_dict = {sorted_nodes.index(n):G.nodes[n]['ccoords']+pillar_oriented_node_coords for n in sorted_nodes}
#reindex the edges in the G with the index in the sorted_nodes
sorted_edges_of_sortednodeidx = [(sorted_nodes.index(e[0]),sorted_nodes.index(e[1])) for e in sorted_edges]

def save_xyz(filename, rotated_positions_dict):
    """
    Save the rotated positions to an XYZ file for visualization.
    """
    with open(filename, "w") as file:
        num_atoms = sum(len(positions) for positions in rotated_positions_dict.values())
        file.write(f"{num_atoms}\n")
        file.write("Optimized structure\n")

        for node, positions in rotated_positions_dict.items():
            for pos in positions:
                file.write(f"X{node}   {pos[0]:.8f} {pos[1]:.8f} {pos[2]:.8f}\n")

# Optimize rotations
num_nodes = G.number_of_nodes()

###3D free rotation
#optimized_rotations,static_xxxx_positions = optimize_rotations(num_nodes,sorted_edges, xxxx_positions_dict)
###2D axis rotation
axis = pillar_vec  # Rotate around x-axis
optimized_rotations = axis_optimize_rotations(axis, num_nodes, G,sorted_nodes,sorted_edges_of_sortednodeidx, xxxx_positions_dict)

# Apply rotations
rotated_node_positions = apply_rotations_to_atom_positions(optimized_rotations, G, sorted_nodes,chain_node_positions_dict)

# Save results to XYZ
save_xyz("optimized_nodesstructure.xyz", rotated_node_positions)

rotated_xxxx_positions_dict,optimized_pair=apply_rotations_to_xxxx_positions(optimized_rotations, G, sorted_nodes, sorted_edges_of_sortednodeidx,xxxx_positions_dict)


start_node = sorted_edges[0][0]#find_nearest_node_to_beginning_point(G)
#loop all of the edges in G and get the lengths of the edges, length is the distance between the two nodes ccoords
edge_lengths,lengths = get_edge_lengths(G)

constant_length = 1.6
x_com_length = np.mean([np.linalg.norm(i) for i in pillar_oriented_node_xcoords])
new_edge_length = linker_length+2*constant_length+2*x_com_length
#update the node ccoords in G by loop edge, start from the start_node, and then update the connected node ccoords by the edge length, and update the next node ccords from the updated node


updated_ccoords,original_ccoords = update_node_ccoords(G,edge_lengths,start_node,new_edge_length)
updated_ccoords,original_ccoords
#exclude the start_node in updated_ccoords and original_ccoords
updated_ccoords = {k:v for k,v in updated_ccoords.items() if k!=start_node}
original_ccoords = {k:v for k,v in original_ccoords.items() if k!=start_node}


#use optimized_params to update all of nodes ccoords in G, according to the fccoords

optimized_params = optimize_cell_parameters(cell_info,original_ccoords,updated_ccoords)
sG,scaled_ccoords = update_ccoords_by_optimized_cell_params(G,optimized_params)
scaled_chain_node_positions_dict = {sorted_nodes.index(n):sG.nodes[n]['ccoords']+pillar_oriented_node_coords for n in sorted_nodes}
scaled_xxxx_positions_dict = {sorted_nodes.index(n):addidx(sG.nodes[n]['ccoords']+pillar_oriented_node_xcoords) for n in sorted_nodes}

# Apply rotations
scaled_rotated_chain_node_positions = apply_rotations_to_atom_positions(optimized_rotations, sG, sorted_nodes,scaled_chain_node_positions_dict)
scaled_rotated_xxxx_positions,optimized_pair = apply_rotations_to_xxxx_positions(optimized_rotations, sG,sorted_nodes, sorted_edges_of_sortednodeidx, scaled_xxxx_positions_dict)
# Save results to XYZ
save_xyz("scale_optimized_nodesstructure.xyz", scaled_rotated_chain_node_positions)

placed_node,placed_edge = place_edgeinnodeframe(sorted_nodes,optimized_pair,node_atom,linker_atom,linker_x_vecs,linker_ccoords,scaled_rotated_xxxx_positions,scaled_rotated_chain_node_positions)

In [ ]:
template_cif_file ='MIL53templatecif.cif'

vvnode,cell_info,unit_cell = extract_bridge_point_cluster_center_from_templatecif(template_cif_file, 'Al',2,3.32) # node com in template cif file, use fcluster to find cluster and the center of the cluster
eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'XX',1,1) # edge com in template cif file, use fcluster to find the cluster and center of the cluster

#loop over super333xxnode and super333yynode to find the pair of x node in unicell which pass through the yynode
vvnode333 = make_supercell_3x3x3(vvnode)
eenode333 = make_supercell_3x3x3(eenode)
pair_vv_e,_,G=find_pair_v_e(vvnode333,eenode333)
G = add_ccoords(G,unit_cell)
G = set_DV_V(G)
#G = set_DE_E(G)
vvnode,eenode

In [ ]:
template_cif_file ='cau-1.cif'

vvnode,cell_info,unit_cell = extract_bridge_point_cluster_center_from_templatecif(template_cif_file, 'Al',2,3.6) # node com in template cif file, use fcluster to find cluster and the center of the cluster
#eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'XX',1,1) # edge com in template cif file, use fcluster to find the cluster and center of the cluster

#loop over super333xxnode and super333yynode to find the pair of x node in unicell which pass through the yynode
#vvnode333 = make_supercell_3x3x3(vvnode)
#eenode333 = make_supercell_3x3x3(eenode)
#pair_vv_e,_,G=find_pair_v_e(vvnode333,eenode333)
#G = add_ccoords(G,unit_cell)
#G = set_DV_V(G)
##G = set_DE_E(G)
#vvnode,eenode
vvnode

In [ ]:
template_cif_file ='MIL53templatecif.cif'

vvnode,cell_info,unit_cell = extract_cluster_center_from_templatecif(template_cif_file, 'YY',1) # node com in template cif file, use fcluster to find cluster and the center of the cluster
eenode,_,_ = extract_cluster_center_from_templatecif(template_cif_file, 'XX',1) # edge com in template cif file, use fcluster to find the cluster and center of the cluster

#loop over super333xxnode and super333yynode to find the pair of x node in unicell which pass through the yynode
vvnode333 = make_supercell_3x3x3(vvnode)
eenode333 = make_supercell_3x3x3(eenode)
pair_vv_e,_,G=find_pair_v_e(vvnode333,eenode333)
G = add_ccoords(G,unit_cell)
G = set_DV_V(G)
G = set_DE_E(G)
vvnode,eenode

In [ ]:
placed_edge.keys()

In [ ]:

# Save results to XYZ
placed_all = []
with open("placed_structure.xyz", "w") as file:
    #node_num_atoms = sum(len(positions) for positions in scaled_rotated_chain_node_positions.values())
    node_num_atoms = sum(len(positions) for positions in placed_node.values())
    edge_num_atoms = sum(len(positions) for positions in placed_edge.values())
    num_atoms = node_num_atoms + edge_num_atoms
    file.write(f"{num_atoms}\n")
    file.write("Optimized structure\n")
    edge_idx = -2
    for node, positions in placed_node.items():
        if 'DV' in G.nodes[node]['type']:
            continue
        for pos in positions:
            file.write(f"{pos[0]}   {pos[1]:.8f} {pos[2]:.8f} {pos[3]:.8f}\n")
            #if pos[0]=='X':
            #    pos[0] = 'C'
            line = np.array([pos[0],pos[1],pos[2],pos[3],0,pos[0],node,'NODE'])
            placed_all.append(line)

    for edge,positions in placed_edge.items():
        
        if 'DE' in G.edges[edge]['type']:
           # pass
            continue
        for pos in positions:
            file.write(f"{pos[0]}   {pos[1]:.8f} {pos[2]:.8f} {pos[3]:.8f}\n")
            #if pos[0]=='X':
               # pos[0] = 'C'
            line = np.array([pos[0],pos[1],pos[2],pos[3],0,pos[0],edge_idx,'EDGE'])
            placed_all.append(line)
        edge_idx -= 1
            #placed_all.append(pos)

end_time =  time.time()
print('time cost:',end_time-start_time)


sc_unit_cell = unit_cell_to_cartesian_matrix(optimized_params[0],optimized_params[1],optimized_params[2],optimized_params[3],optimized_params[4],optimized_params[5])

def write_cif_nobond(placed_all, scaled_params, cifname):
	sc_a,sc_b,sc_c,sc_alpha,sc_beta,sc_gamma = scaled_params
	os.makedirs('output_cifs',exist_ok=True)

	opath = os.path.join('output_cifs', cifname)
	
	with open(opath, 'w') as out:
		out.write('data_' + cifname[0:-4] + '\n')
		out.write('_audit_creation_date              ' + datetime.datetime.today().strftime('%Y-%m-%d') + '\n')
		out.write("_audit_creation_method            'MOFbuilder_1.0'" + '\n')
		out.write("_symmetry_space_group_name_H-M    'P1'" + '\n')
		out.write('_symmetry_Int_Tables_number       1' + '\n')
		out.write('_symmetry_cell_setting            triclinic' + '\n')
		out.write('loop_' + '\n')
		out.write('_symmetry_equiv_pos_as_xyz' + '\n')
		out.write('  x,y,z' + '\n')
		out.write('_cell_length_a                    ' + str(sc_a) + '\n')
		out.write('_cell_length_b                    ' + str(sc_b) + '\n')
		out.write('_cell_length_c                    ' + str(sc_c) + '\n')
		out.write('_cell_angle_alpha                 ' + str(sc_alpha) + '\n')
		out.write('_cell_angle_beta                  ' + str(sc_beta) + '\n')
		out.write('_cell_angle_gamma                 ' + str(sc_gamma) + '\n')
		out.write('loop_' + '\n')
		out.write('_atom_site_label' + '\n')
		out.write('_atom_site_type_symbol' + '\n')
		out.write('_atom_site_fract_x' + '\n')
		out.write('_atom_site_fract_y' + '\n')
		out.write('_atom_site_fract_z' + '\n')
		

		for l in placed_all:

			vec = list(map(float, l[1:4]))
			cvec = np.dot(np.linalg.inv(sc_unit_cell), vec)
			cvec = np.mod(cvec, 1) # makes sure that all fractional coordinates are in [0,1]
			out.write('{:7} {:>4} {:>15} {:>15} {:>15}'.format(l[0], re.sub('[0-9]','',l[0]), "%.10f" % np.round(cvec[0],10), "%.10f" % np.round(cvec[1],10), "%.10f" % np.round(cvec[2],10)))
			out.write('\n')

		out.write('loop_' + '\n')
		out.write('_geom_bond_atom_site_label_1' + '\n')
		out.write('_geom_bond_atom_site_label_2' + '\n')
		out.write('_geom_bond_distance' + '\n')
		out.write('_geom_bond_site_symmetry_2' + '\n')
		out.write('_ccdc_geom_bond_type' + '\n')
write_cif_nobond(placed_all, optimized_params, 'placed_structure.cif')

In [157]:
import os
import networkx as nx
import numpy as np
import re
import glob
import py3Dmol as p3d
from tests.MOF_builder.functions.ciftemplate2graph import ct2g
from tests.MOF_builder.functions.remove_dummy_atoms import remove_Fr
from tests.MOF_builder.functions.adjust_edges import adjust_edges,addxoo2edge,superimpose
from tests.MOF_builder.functions.write_cifs import write_cif_nobond, merge_catenated_cifs #write_check_cif,bond_connected_components, distance_search_bond, fix_bond_sym, merge_catenated_cifs
#from functions.scale_animation import scaling_callback_animation, write_scaling_callback_animation, animate_objective_minimization
import itertools
#from random import choice
from tests.MOF_builder.functions.cluster import cluster_supercell,placed_arr
from tests.MOF_builder.functions.supercell import find_new_node_beginning,Carte_points_generator
from tests.MOF_builder.functions.output import tempgro,temp_xyz,viewgro#,viewxyz
from tests.MOF_builder.functions.terminations import terminate_nodes,terminate_unsaturated_edges,add_node_terminations,exposed_xoo_cc,Xpdb,terminate_unsaturated_edges_CCO2
#from functions.filtX import filt_nodex_fvec,filt_closest_x_angle,filt_outside_edgex
#unctions.multitopic import 
from tests.MOF_builder.functions.isolated_node_cleaner import reindex_frag_array,get_frag_centers_fc,calculate_eG_net_ditopic,calculate_eG_net_ditopic_PILLAR,calculate_eG_net
from tests.MOF_builder.functions.replace import fetch_by_idx_resname,sub_pdb


####### Global options #######


pi = np.pi

vname_dict = {'V':1,'Er':2,'Ti':3,'Ce':4,'S':5,
			  'H':6,'He':7,'Li':8,'Be':9,'B':10,
			  'C':11,'N':12,'O':13,'F':14,'Ne':15,
			  'Na':16,'Mg':17,'Al':18,'Si':19,'P':20 ,
			  'Cl':21,'Ar':22,'K':23,'Ca':24,'Sc':24,
			  'Cr':26,'Mn':27,'Fe':28,'Co':29,'Ni':30}

metal_elements = ['Ac','Ag','Al','Am','Au','Ba','Be','Bi',
				  'Bk','Ca','Cd','Ce','Cf','Cm','Co','Cr',
				  'Cs','Cu','Dy','Er','Es','Eu','Fe','Fm',
				  'Ga','Gd','Hf','Hg','Ho','In','Ir',
				  'K','La','Li','Lr','Lu','Md','Mg','Mn',
				  'Mo','Na','Nb','Nd','Ni','No','Np','Os',
				  'Pa','Pb','Pd','Pm','Pr','Pt','Pu','Ra',
				  'Rb','Re','Rh','Ru','Sc','Sm','Sn','Sr',
				  'Ta','Tb','Tc','Th','Ti','Tl','Tm','U',
				  'V','W','Y','Yb','Zn','Zr']




class MOF_ditopic:
    #def __init__(self,template,node,edge,node_topics):
    #    self.template = template
    #    self.node = node
    #    self.edge = edge
    #    self.linker_topics = 2 #ditopic class 
    #    self.node_topics = node_topics
	
    def __init__(self):
        self.linker_topics = 2
        pass
    def load(self,TG,placed_all,sc_unit_cell,placed_nodes,placed_edges):
        self.TG = TG
        self.placed_all = placed_all
        self.sc_unit_cell = sc_unit_cell
        self.placed_nodes = placed_nodes
        self.placed_edges = placed_edges

    def basic_supercell(self,supercell,term_file = 'data/methyl.pdb',boundary_cut_buffer = 0.00,edge_center_check_buffer = 0.0,cutxyz=[True,True,True]):
        linker_topics = self.linker_topics
        cutx,cuty,cutz = cutxyz
        TG = self.TG
        scalar = boundary_cut_buffer
        boundary_scalar = edge_center_check_buffer
        placed_edges = self.placed_edges
        placed_nodes = self.placed_nodes
        sc_unit_cell = self.sc_unit_cell
		
        frame_node_name= list(TG.nodes())
        frame_node_fc=np.asarray([TG.nodes[fn]['fcoords']for fn in frame_node_name])
    
        #new_beginning_fc = find_new_node_beginning(frame_node_fc)		
        new_beginning_fc = np.array([0,0,0])
        placed_nodes_arr,nodes_id=placed_arr(placed_nodes)
        placed_edges_arr,edges_id=placed_arr(placed_edges)		
        placed_nodes_fc = np.hstack((placed_nodes_arr[:,0:1],np.dot(np.linalg.inv(sc_unit_cell),placed_nodes_arr[:,1:4].T).T-new_beginning_fc,placed_nodes_arr[:,4:]))
        placed_edges_fc = np.hstack((placed_edges_arr[:,0:1],np.dot(np.linalg.inv(sc_unit_cell),placed_edges_arr[:,1:4].T).T-new_beginning_fc,placed_edges_arr[:,4:]))		
			
        target_all_fc = np.vstack((placed_nodes_fc,placed_edges_fc))
        #target_all_fc = np.vstack((placed_nodes_fc,tetratopic_edges_fcoords)) # the reason for use above version node is because we need xoo in node for terminations adding
        box_bound= supercell+1
        supercell_Carte = Carte_points_generator(supercell)		
        connected_nodeedge_fc, boundary_connected_nodes_res,eG,bare_nodeedge_fc_loose=cluster_supercell(sc_unit_cell,supercell_Carte,linker_topics,target_all_fc,box_bound,scalar,cutx,cuty,cutz,boundary_scalar)		
        terms_cc_loose = terminate_nodes(term_file,boundary_connected_nodes_res,connected_nodeedge_fc,sc_unit_cell,box_bound)

        connected_nodeedge_cc = np.hstack((connected_nodeedge_fc[:,:-3],np.dot(sc_unit_cell,connected_nodeedge_fc[:,-3:].T).T))
        #print(connected_nodeedge_cc.shape,terms_cc_loose.shape)

        node_edge_term_cc_loose = np.vstack((connected_nodeedge_cc,terms_cc_loose))		
        self.all_connected_node_edge_cc = connected_nodeedge_cc
        self.all_terms_cc_loose = terms_cc_loose
        self.all_N_E_T_cc = node_edge_term_cc_loose		
        self.bare_nodeedge_fc = bare_nodeedge_fc_loose

    def write_basic_supercell(self,gro,xyz):
        tempgro('20test.gro',self.all_connected_node_edge_cc[self.all_connected_node_edge_cc[:,5]==1])
        tempgro(gro,self.all_N_E_T_cc)
        temp_xyz(xyz,self.all_N_E_T_cc)
        viewgro("20test.gro")
        viewgro(gro)
	
	
    '''
        #Defective model: node/edge missing
            #delete nodes or edges
            #terminate nodes ROUND1
            # find main fragment
                # find unstaturated node uN1
                    # find uN1 neighbors and extract X in neighbor edge(E+int)
                        # filt exposed X sites in uN1
                            # add terminations

            #terminate edge ROUND2
                #find unsaturaed edge uE1
                    #find uE1 neighbors and extract X in neighbor node 'int'
                        # filt exposed X sites in uE1
                            # add terminations (-OH)

        #Defective model: linker exchange
        #   termination OO don't change, use X to set a range 
        #   atoms in outX_range stay
        #    then superimpose by X


            #superimpose for replacement
                #find X AND super impose
    '''
    def defect_missing(self,remove_node_list = [],remove_edge_list = []):
        bare_nodeedge_fc_loose = self.bare_nodeedge_fc
        linker_topics = self.linker_topics
        node_topics = self.node_topics
        sc_unit_cell =self.sc_unit_cell

        renode1_fcarr=reindex_frag_array(bare_nodeedge_fc_loose,'NODE')
        reedge1_fcarr=reindex_frag_array(bare_nodeedge_fc_loose,'EDGE')
        defective_node_fcarr = np.vstack(([i for i in renode1_fcarr if i[5] not in remove_node_list]))
        defective_edge_fcarr = np.vstack(([i for i in reedge1_fcarr if i[5] not in remove_edge_list]))
        renode_fcarr = reindex_frag_array(defective_node_fcarr,'NODE')
        reedge_fcarr = reindex_frag_array(defective_edge_fcarr,'EDGE')
        edgefc_centers = get_frag_centers_fc(reedge_fcarr)
        nodefc_centers = get_frag_centers_fc(renode_fcarr)

        #eG = calculate_eG_net_ditopic(edgefc_centers,nodefc_centers,linker_topics)
        eG = calculate_eG_net_ditopic_PILLAR(edgefc_centers,nodefc_centers,linker_topics,1)
        eG_subparts=[len(c) for c in sorted(nx.connected_components(eG), key=len, reverse=True)]
        #for pillar stack, add virtual edge to connect clostest nodes, set distance to 0.6
        if len(eG_subparts)>1:
            print(f'this MOF has {len(eG_subparts)} seperated fragments: {eG_subparts}')
        else:
            print(f'this MOF has {len(eG_subparts)} fragment')



        frags=[(len(c),c) for c in sorted(nx.connected_components(eG), key=len, reverse=True)]
        main_frag=list(sorted(nx.connected_components(eG), key=len, reverse=True)[0]) 
       

        main_frag_nodes = [i for i in main_frag if isinstance(i,int)]
        main_frag_edges = [i for i in main_frag if re.sub('[0-9]','',str(i)) == 'E']



        #delete "virtual" edges
        for edge_n in eG.edges():
            if eG.edges[edge_n]['type'] == 'virtual':
                eG.remove_edge(edge_n[0],edge_n[1])

        unsaturated_nodes = [(n,d) for n, d in eG.degree() if d <node_topics and isinstance(n,int)]
        unsaturated_edges = [(n,d) for n, d in eG.degree() if d <linker_topics and isinstance(n,str)]
        


        if len(unsaturated_edges) > 0 :
            print(f"UNsaturated edges(linkers) exist, need linker_termination <= {len(unsaturated_edges)}")
        else:
            print("only saturated edges(linkers) exist")

        if len(unsaturated_nodes) > 0 :
            print(f"UNsaturated nodes exist, <={len(unsaturated_nodes)} nodes need node_termination")
            print(unsaturated_nodes)
        else:
            print("only saturated nodes exist")



        unsaturated_main_frag_nodes = [i for i in unsaturated_nodes if i[0] in main_frag_nodes]
        unsaturated_main_frag_edges = [i for i in unsaturated_edges if i[0] in main_frag_edges]

        main_frag_half_edges_fc = np.vstack(([reedge_fcarr[reedge_fcarr[:,5]==int(ei[1:])]for ei in main_frag_edges]))
        main_frag_nodes_fc = np.vstack(([renode_fcarr[renode_fcarr[:,5]==ni]for ni in main_frag_nodes]))
        main_frag_edges_fc,xoo_dict,con_nodes_x_dict = addxoo2edge(eG,main_frag_nodes,main_frag_nodes_fc,main_frag_edges,main_frag_half_edges_fc,sc_unit_cell)

        main_frag_nodes_cc = np.hstack((main_frag_nodes_fc[:,:-3],np.dot(main_frag_nodes_fc[:,-3:],sc_unit_cell)))
        main_frag_edges_cc = np.hstack((main_frag_edges_fc[:,:-3],np.dot(main_frag_edges_fc[:,-3:],sc_unit_cell)))
        self.eG = eG
        self.main_frag_nodes = main_frag_nodes
        self.main_frag_edges = main_frag_edges
        self.unsaturated_main_frag_nodes = unsaturated_main_frag_nodes
        self.unsaturated_main_frag_edges = unsaturated_main_frag_edges
        self.node_xoo_dict = xoo_dict
        self.con_nodes_x_dict = con_nodes_x_dict
        self.main_frag_nodes_cc = main_frag_nodes_cc
        self.main_frag_edges_cc = main_frag_edges_cc   

    def term_defective_model(self,n_term_file = 'data/methyl.pdb',e_termfile = 'data/CCO2.pdb'):
        eG = self.eG
        unsaturated_main_frag_nodes = self.unsaturated_main_frag_nodes
        main_frag_nodes = self.main_frag_nodes
        main_frag_nodes_cc =self.main_frag_nodes_cc
        con_nodes_x_dict =self.con_nodes_x_dict
        xoo_dict = self.node_xoo_dict
        unsaturated_main_frag_edges = self.unsaturated_main_frag_edges
        main_frag_edges_cc = self.main_frag_edges_cc
        linker_topics = self.linker_topics
		
		# get indices for cleaved node atoms in main_frag_nodes  without xoo

        xoo_ind = []
        for key in list(xoo_dict):
            xoo_ind.append(key)
            xoo_ind += xoo_dict[key]
        single_node = main_frag_nodes_cc[main_frag_nodes_cc[:,5]==main_frag_nodes_cc[0,5]]
        node_nums = len(single_node)
        single_node_stay_ind=np.asarray([i for i in range(node_nums) if i not in xoo_ind])
        a =[]
        for node in range(len(main_frag_nodes)):
            a.append(node_nums*node+single_node_stay_ind)
        metal_node_indices = np.hstack(a)

        #add -methyl to terminate nodes find exposed xoo 
        ex_node_cxo_cc_loose=exposed_xoo_cc(eG,unsaturated_main_frag_nodes,main_frag_nodes_cc,con_nodes_x_dict,xoo_dict)
        n_terms_loose = add_node_terminations(n_term_file,ex_node_cxo_cc_loose)
        if len(n_terms_loose) > 0:
            n_terms_cc = np.vstack((n_terms_loose))
            self.n_terms_cc = n_terms_cc
        else:
            n_terms_cc = np.empty((0,9))
            self.n_terms_cc = n_terms_cc

        #add -COOH term to exposed edge and change edge name to HEDGE
        if os.path.basename(e_termfile)=='CCO2.pdb':
            cleaved_metal_node = main_frag_nodes_cc[metal_node_indices]
            t_edges = terminate_unsaturated_edges_CCO2(e_termfile,unsaturated_main_frag_edges,eG,main_frag_edges_cc,linker_topics)
            node_edge_term_cc= np.vstack((cleaved_metal_node,t_edges,n_terms_cc))
        else:
            cleaved_metal_node = main_frag_nodes_cc[metal_node_indices]
            t_edges = terminate_unsaturated_edges(e_termfile,unsaturated_main_frag_edges,eG,main_frag_edges_cc,linker_topics)
            node_edge_term_cc= np.vstack((cleaved_metal_node,t_edges,n_terms_cc))
        self.t_edges = t_edges
        self.tn_te_cc = node_edge_term_cc
			


    def write_tntemof(self,gro):
        tempgro(gro,self.tn_te_cc)
        #temp_xyz("303term_supercell.xyz",self.tn_te_cc)
        viewgro(gro)


    def defect_replace_linker(self,sub_file,sub_class,candidate_res_idx_list,sub_res_newname = 'SUB'):
        node_edge_term_cc = self.tn_te_cc
        for res_idx in candidate_res_idx_list:
            fetch_res_mask,other_mask = fetch_by_idx_resname(node_edge_term_cc,res_idx,sub_class)
            other_res = node_edge_term_cc[other_mask]
            selected_res = node_edge_term_cc[fetch_res_mask]
            if len(selected_res)==0:
                continue
            X_atoms_ind = [i for i in range(len(selected_res)) if selected_res[i,2][0] == 'X']
            outer_atoms_ind = [j for j in range(len(selected_res)) if (j > X_atoms_ind[0]) & (j not in X_atoms_ind)]
            stay_outer_atoms = selected_res[outer_atoms_ind]
            X_atoms = selected_res[X_atoms_ind]
            sub_data=sub_pdb(sub_file)
            subX,subX_ind= Xpdb(sub_data,'X')
            subX_coords_cc = subX[:,-3:]
            subX_coords_cc = subX_coords_cc.astype('float')
            X_atoms_coords_cc = X_atoms[:,-3:]
            X_atoms_coords_cc =  X_atoms_coords_cc.astype('float')

            _,rot,trans = superimpose(subX_coords_cc,X_atoms_coords_cc)
            sub_coords = sub_data[:,-3:]
            sub_coords = sub_coords.astype('float')
            placed_sub_data = np.hstack((sub_data[:,:-3],np.dot(sub_coords,rot)+trans))

            sub_edge = np.vstack((stay_outer_atoms,placed_sub_data))
            sub_edge[:,5]= stay_outer_atoms[0,5]
            sub_edge[:,4] = sub_res_newname
            for row in range(len(sub_edge)):
                sub_edge[row,2] = re.sub('[0-9]','',sub_edge[row,2])+str(row+1)
            node_edge_term_cc = np.vstack((other_res,sub_edge))
        self.replaced_tn_te_cc = node_edge_term_cc
    def write_view_replaced(self,gro):
        tempgro(gro,self.replaced_tn_te_cc)
        viewgro(gro)
        

In [26]:
placed_all = np.array(placed_all)
placed_N = placed_all[placed_all[:,7]=='NODE']
placed_E = placed_all[placed_all[:,7]=='EDGE']


In [ ]:

n_term_file = 'data/terminations_database/methyl.pdb'
e_termfile = 'data/terminations_database/CCO2.pdb'

supercell = [2,1,1]

mof = MOF_ditopic()
mof.node_topics = 2
mof.load(sG,placed_all,sc_unit_cell,placed_N,placed_E)
mof.basic_supercell(np.asarray(supercell),term_file = 'data/terminations_database/methyl.pdb',boundary_cut_buffer = 0,edge_center_check_buffer = 0.50,cutxyz=[True,True,True])
mof.write_basic_supercell('20test.gro','20test.xyz')
mof.node_topics = 2
mof.defect_missing()
mof.term_defective_model(n_term_file=n_term_file,e_termfile=e_termfile)
mof.write_tntemof('30.gro')    
#  
#'''cleave all unsaturated linkers'''
u_edge_idx=[]
for i in mof.unsaturated_main_frag_edges:
    u_edge_idx.append(int(i[0][1:]))
print(u_edge_idx)

mof.defect_missing([],u_edge_idx)#+[26,20,18,30,8,34,33,4,28,29])
mof.term_defective_model(n_term_file=n_term_file,e_termfile=e_termfile)
mof.write_tntemof('33.gro') 
 